# Lisbon Machine Learning School
## Exercise 2: Classification, showcase with CMS S-top Search: Signal events vs Background events

(C) Cristóvão B. da Cruz e Silva (Laboratório de Instrumentação e Física Experimental de Partículas), cbeiraod@cern.ch


Dislcaimer: some content and examples shamelessly "stolen" from Pietro Vischia's Exercise 2

## Setup the environment

- If you are running locally, you don't need to run anything, but make sure to point the base_data_dir variable to the directory where you have the data

- If you are running on Google Colab, uncomment and run the next cell (remove only the "#", keep the "!"). You can also run it from a local installation, but it will do nothing if you have already installed all dependencies (and it will take some time to tell you it is not gonna do anything).



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/"
#! git clone https://github.com/vischia/lisbon-ml-school.git
#%cd lisbon-ml-school
#!pwd
#!ls

This exercise follows from the CMS analysis [https://doi.org/10.1007/JHEP09(2018)065][https://arxiv.org/abs/1805.05784] in search of pair production of the supersymmetric partner of the top quark in the situation where the difference in mass between the s-top quark and the lightest supersymmetric particle (LSP) is smaller than the W mass, in this situation the s-top quark undergoes a "four-body decay".

![StopProduction](figs/StopProd.png)

The paper above focuses on the data from 2016, while the analysis was later extended to the full Run II dataset, [https://doi.org/10.1007/JHEP06(2023)060][https://arxiv.org/abs/2301.08096].

Both papers use a BDT machine learning algorithm for classifying background and signal events. One BDT is trained for each $\Delta M$ region, where $\Delta M$ is defined as the difference in mass between the s-top and the LSP, typically a neutralino. As a result, 8 different BDTs are trained for these analysis, for each year of data taking (the years are treated independently).

SUSY models are characterised by large phase spaces which must be excluded. For these analysis a simplified model spectra model for the signal samples were used, in these models the SM is extended only by the minimal particles and interactions thought to be of relevance for what is under study, in this case the s-top and the LSP as well as their corresponding vertices in the diagram above. This still provides quite a large phase space for the analysis search, in particular the phase space covers possible masses for the LSP vs possible masses for the s-top. As a result, merging of the signal samples is desirable to reduce the analysis complexity. It can also be observed that to first order, the kinematics of the decay products depend on the $\Delta M$ quantity, so splitting by $\Delta M$ is a reasonable approach.

Given the small mass difference between the s-top and the LSP, the decay prodcuts of this channel are typically quite soft and may not fully pass strict selection criteria. For this reason the analysis requires the event to have one hard jet, this jet is interpreted to be a jet originating from Initial State Radiation (ISR) and provides a boost to the decay products of the s-top decay. For this same reason the event selection only requires one lepton and one jet from the event together with missing transverse energy, the presence of b-jets, while necessary according to the diagram is not enforced due to the low efficiency. However, the classifier of the jet with the highest b-tag is passed to the machine learning algorithm in order to provide information about the b-jet content to the ML approach. The preselection is mostly uniform for all $\Delta M$ regions, with only $\Delta M = {70,80}$ requiring a slightly tuned preselection to account for the more challenging kinematics in this region.

An iterative procedure was used in the development of these analysis and a subset of 12 input features to the BDT were observed to deliver the best performance. The variable are:
 - Jet1Pt - Transverse momentum of the leading jet
 - mt - Transverse mass
 - Met - Missing transverse energy
 - LepChg - Charge of the leading lepton
 - LepEta - Eta of the leading lepton
 - LepPt - Transverse momentum of the leading lepton
 - HT - sum of the transverse momentum of all particles in the event
 - NbLoose - number of jets passing loose b-jet criteria
 - Njet - number of jets
 - JetHBpt - Transverse momentum of the jet with the highest b-tag score
 - DrJetHBLep - $\Delta R$ between the jet with the highest b-tag score and the leading lepton
 - JetHBCSV - B-tag score of the jet with the highest b-tag score

Included in the nTuples are several other variables considered to be possibly useful for the analysis, an invested individual could explore the use of these other features in addition or in alternative to the ones aboves.

In this exercise we will reimplement this analysis using a Neural Network algorithm and compare the performance to that of the BDT approach used in the papers. The exact same BDT classifiers used in the 2016 analysis were applied to these MC samples and the output is stored in a separate nTuple and loaded as an additional column, providing us easy access to perform a preliminary comparison.

## Fetch the data

If you have not yet retrieved the data and placed it into an appropriate folder you can access either from google drive or on your local computer, follow the equivalent steps below.

The steps below should work for colab without any modification, you will need to modify them to run locally on your machine.

In [ ]:
import os
# To run locally:
if not os.path.isfile("/content/drive/MyDrive/") and not os.path.isdir("data/Lisbon_ML_School_Stop"):
    !cd data/; wget https://cernbox.cern.ch/remote.php/dav/public-files/fBw0wnuLt1CWFGa/Lisbon_ML_School_Stop.tar.gz; tar xzvf Lisbon_ML_School_Stop.tar.gz; cd -;
# To run in colab:
if os.path.isfile("/content/drive/MyDrive/") and not os.path.isfile("/content/drive/MyDrive/Lisbon_ML_School_Stop/Data/ZZ.root"):
    !cd /content/drive/MyDrive; wget https://cernbox.cern.ch/remote.php/dav/public-files/fBw0wnuLt1CWFGa/Lisbon_ML_School_Stop.tar.gz; tar xzvf Lisbon_ML_School_Stop.tar.gz; cd -;

## Load the needed libraries

In [ ]:
if os.path.isdir("/content/drive/MyDrive/"): # Only on colab, locally you should have already installed these packages via conda    
    !pip install torchinfo
    !pip install uproot
    !pip install pyhf

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import torchinfo
from tqdm import tqdm

import sklearn
import sklearn.model_selection
from sklearn.metrics import roc_curve, auc, accuracy_score

import uproot

import pandas

import matplotlib
matplotlib.rcParams['figure.figsize'] = (8, 6)
matplotlib.rcParams['axes.labelsize'] = 14
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

import plotly.express as px

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.set_default_dtype(torch.float32)

print('Using torch version', torch.__version__)

## Load the data

We use the [uproot](https://uproot.readthedocs.io/en/latest/basic.html) library to conveniently read in a [ROOT TNuple](https://root.cern.ch/doc/master/classTNtuple.html) which can automatically convert it to a [pandas dataframe](https://pandas.pydata.org/).

In [ ]:
def load_dataframe(base_dir, dataset, bdt = True, treename = 'data', previous_dataframes = None, do_split = True, train_factor=0.5, seed = None, category=None, subcategory=None):
  if seed is not None:
    np.random.seed(seed = seed)
  if previous_dataframes is not None and dataset in previous_dataframes:
    df = previous_dataframes[dataset]
  elif dataset[-5:] == '.root':
    # Load base data
    df = uproot.open(os.path.join(base_dir, 'Data', dataset))[treename].arrays(library='pd')
    #df = uproot.open(os.path.join(base_dir, 'DataC', dataset))[treename].arrays(library='pd') # DataC may be better optimized for ML, maybe the students can check

    # Load friend trees
    if bdt:
      df_bdt = uproot.open(os.path.join(base_dir, 'BDT', dataset))[treename].arrays(library='pd')
      df = pandas.concat([df,df_bdt],axis=1)
      del df_bdt

    # Apply final selections
    # Preselection is already applied:
    #     "(tightLepton) && (DPhiJet1Jet2 < 2.8 || Jet2Pt < 67) && (HT > 181) && (Met > 254) && (Jet1Pt > 100)",
    # For High DM (only 70 and 80), apply final selection of:
    #     "(DPhiJet1Jet2 < 2.5 || Jet2Pt < 60) && (HT > 200) && (Met > 280) && (Jet1Pt > 110)",
    # For Low DM, apply final selection of:
    #     "(DPhiJet1Jet2 < 2.5 || Jet2Pt < 60) && (HT > 200) && (Met > 280) && (Jet1Pt > 110) && (LepPt < 30)",
    #sel = (df['DPhiJet1Jet2'] < 2.5) | (df['Jet2Pt'] < 60) & (df['HT'] > 200) & (df['Met'] > 280) & (df['Jet1Pt'] > 110)
    sel = (df['DPhiJet1Jet2'] < 2.5) | (df['Jet2Pt'] < 60) & (df['HT'] > 200) & (df['Met'] > 280) & (df['Jet1Pt'] > 110) & (df['LepPt'] < 30)
    df = df.loc[sel]

    # Split train/test
    if do_split:
      2+2
      #df['isTrain'] = np.random.choice([True, False], size=len(df), p=[train_factor, 1-train_factor])

    # Apply ne columns
    if category is not None:
      df['category'] = category
    if subcategory is not None:
      df['subcategory'] = subcategory
  else:
    if len(dataset) > 2 and dataset[:2] == 'DM' and int(dataset[2:]) in [10, 20, 30, 40, 50, 60, 70, 80]:
      dm = int(dataset[2:])
      ds_list = []
      for stopM in range(250, 801, 25):
        neutM = stopM - dm
        ds_list.append((f'T2DegStop_{stopM}_{neutM}.root', f'Signal_{stopM}_{neutM}'))
    elif dataset == 'DYJets':
      ds_list = [
          'DYJetsToLL_M5to50_HT100to200.root',
          'DYJetsToLL_M5to50_HT200to400.root',
          'DYJetsToLL_M5to50_HT400to600.root',
          'DYJetsToLL_M5to50_HT600toInf.root',
          'DYJetsToLL_M50_HT100to200.root',
          'DYJetsToLL_M50_HT200to400.root',
          'DYJetsToLL_M50_HT400to600.root',
          'DYJetsToLL_M50_HT600to800.root',
          'DYJetsToLL_M50_HT800to1200.root',
          'DYJetsToLL_M50_HT1200to2500.root',
          'DYJetsToLL_M50_HT2500toInf.root',
      ]
    elif dataset == 'QCD':
      ds_list = [
          'QCD_HT50to100.root',
          'QCD_HT100to200.root',
          'QCD_HT200to300.root',
          'QCD_HT300to500.root',
          'QCD_HT500to700.root',
          'QCD_HT700to1000.root',
          'QCD_HT1000to1500.root',
          'QCD_HT1500to2000.root',
          'QCD_HT2000toInf.root',
      ]
    elif dataset == 'SingleT':
      ds_list = [
          'T_tch_powheg.root',
          'T_tWch_ext.root',
          'TBar_tch_powheg.root',
          'TBar_tWch_ext.root',
      ]
    elif dataset == 'TT_Pow':
      ds_list = [
          #'TT_pow_backup.root',
          'TT_pow.root',
      ]
    elif dataset == 'TTLep':
      ds_list = [
          'TTJets_DiLepton.root',
          'TTJets_SingleLeptonFromT.root',
          'TTJets_SingleLeptonFromTbar.root',
      ]
    elif dataset == 'TTX':
      ds_list = [
          'TTGJets.root',
          'TTWToLNu.root',
          'TTWToQQ.root',
          'TTZToLLNuNu_m1to10.root',
          'TTZToLLNuNu.root',
          'TTZToQQ.root',
      ]
    elif dataset == 'WJets':
      ds_list = [
          'Wjets_70to100.root',
          'Wjets_100to200.root',
          'Wjets_200to400.root',
          'Wjets_400to600.root',
          'Wjets_600to800.root',
          'Wjets_800to1200.root',
          'Wjets_1200to2500.root',
          'Wjets_2500toInf.root',
      ]
    elif dataset == 'Diboson':
      ds_list = [
          'WW.root',
          'WZ.root',
          'ZZ.root',
      ]
    elif dataset == 'ZInv':
      ds_list = [
          'ZJetsToNuNu_HT100to200.root',
          'ZJetsToNuNu_HT200to400.root',
          'ZJetsToNuNu_HT400to600.root',
          'ZJetsToNuNu_HT600to800.root',
          'ZJetsToNuNu_HT800to1200.root',
          'ZJetsToNuNu_HT1200to2500.root',
          'ZJetsToNuNu_HT2500toInf.root',
      ]
      #'TTW_LO.root',
      #'TTZ_LO.root',
    elif dataset == "Background_TTPow":
      ds_list = [
          'TT_Pow',
          'WJets',
      ]
    elif dataset == "Background_TTPow_ZInv":
      ds_list = [
          'TT_Pow',
          'WJets',
          'ZInv',
      ]
    elif dataset == "Background_TTLep":
      ds_list = [
          'TTLep',
          'WJets',
      ]
    elif dataset == "Background_TTLep_ZInv":
      ds_list = [
          'TTLep',
          'WJets',
          'ZInv',
      ]
    else:
      raise RuntimeError(f"Unable to find dataset {dataset}")
    df = None
    for ds in ds_list:
      if isinstance(ds, tuple):
        subcat = ds[1]
        ds = ds[0]
      else:
        subcat = subcategory
      new_df = load_dataframe(base_dir, ds, bdt = bdt, treename = treename, previous_dataframes = previous_dataframes, category=category, subcategory=subcat)
      if previous_dataframes is not None:
        if ds not in previous_dataframes:
          previous_dataframes[ds] = new_df
      if df is None:
        df = new_df
      else:
        df = pandas.concat([df, new_df], ignore_index=True)
        if previous_dataframes is None:
          del new_df
  #df.set_index('Event', inplace=True)
  return df

def load_all_dataframes(base_dir, bdt = True, treename = 'data', seed = None):
  if seed is not None:
    np.random.seed(seed = seed)

  dataframes = {}

  # Signal samples per \DeltaM
  #dataframes['DM10'] = load_dataframe(base_dir, "DM10", bdt = bdt, treename = treename, previous_dataframes = dataframes)
  #dataframes['DM20'] = load_dataframe(base_dir, "DM20", bdt = bdt, treename = treename, previous_dataframes = dataframes)
  dataframes['DM30'] = load_dataframe(base_dir, "DM30", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Signal")
  #dataframes['DM40'] = load_dataframe(base_dir, "DM40", bdt = bdt, treename = treename, previous_dataframes = dataframes)
  #dataframes['DM50'] = load_dataframe(base_dir, "DM50", bdt = bdt, treename = treename, previous_dataframes = dataframes)
  #dataframes['DM60'] = load_dataframe(base_dir, "DM60", bdt = bdt, treename = treename, previous_dataframes = dataframes)
  #dataframes['DM70'] = load_dataframe(base_dir, "DM70", bdt = bdt, treename = treename, previous_dataframes = dataframes)
  #dataframes['DM80'] = load_dataframe(base_dir, "DM80", bdt = bdt, treename = treename, previous_dataframes = dataframes)

  # Example points from different DM
  # DM 10
  dataframes['T2DegStop_250_240.root'] = load_dataframe(base_dir, "T2DegStop_250_240.root", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Signal", subcategory="Signal_250_240")
  # DM 30
  dataframes['T2DegStop_250_220.root'] = load_dataframe(base_dir, "T2DegStop_250_220.root", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Signal", subcategory="Signal_250_220")
  # DM 50
  dataframes['T2DegStop_250_200.root'] = load_dataframe(base_dir, "T2DegStop_250_200.root", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Signal", subcategory="Signal_250_200")
  # DM 80
  dataframes['T2DegStop_250_170.root'] = load_dataframe(base_dir, "T2DegStop_250_170.root", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Signal", subcategory="Signal_250_170")

  # Individual MC samples
  dataframes['DYJets'] = load_dataframe(base_dir, "DYJets", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="DYJets")
  dataframes['QCD'] = load_dataframe(base_dir, "QCD", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="QCD")
  dataframes['SingleT'] = load_dataframe(base_dir, "SingleT", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="SingleTop")
  dataframes['TT_Pow'] = load_dataframe(base_dir, "TT_Pow", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="TT")
  dataframes['TTLep'] = load_dataframe(base_dir, "TTLep", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="TT")
  dataframes['TTX'] = load_dataframe(base_dir, "TTX", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="TTX")
  dataframes['WJets'] = load_dataframe(base_dir, "WJets", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="WJets")
  dataframes['Diboson'] = load_dataframe(base_dir, "Diboson", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="VV")
  dataframes['ZInv'] = load_dataframe(base_dir, "ZInv", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="ZInv")

  # Choose your favorite background model, but only 1
  #dataframes['Background'] = load_dataframe(base_dir, "Background_TTPow", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="Other")
  #dataframes['Background'] = load_dataframe(base_dir, "Background_TTPow_ZInv", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="Other")
  dataframes['Background'] = load_dataframe(base_dir, "Background_TTLep", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="Other")
  #dataframes['Background'] = load_dataframe(base_dir, "Background_TTLep_ZInv", bdt = bdt, treename = treename, previous_dataframes = dataframes, category="Background", subcategory="Other")

  return dataframes

In [ ]:
base_data_dir=None

# To run locally:
if not os.path.isfile("/content/drive/MyDrive/") and not os.path.isdir("data/Lisbon_ML_School_Stop"):
    base_data_dir = 'data/Lisbon_ML_School_Stop'
if os.path.isfile("/content/drive/MyDrive/") and not os.path.isfile("/content/drive/MyDrive/Lisbon_ML_School_Stop/Data/ZZ.root"):
    base_data_dir = '/content/drive/MyDrive/Lisbon_ML_School_Stop/Data'


dataframes = load_all_dataframes(base_data_dir, seed = 42)

In [ ]:
list(dataframes.keys())

## Inspect the data

Before applying any "fancy" methods to the data, it is better to understand what the data is and looks like. **Spend some time exploring the data**.


We have loaded several dataframes with simulated data for different processes, of particular interest to us are the signal dataframes and background dataframe. All dataframes have the same structure, let's start by looking at the structure of one of the dataframes:
   

In [ ]:
print(dataframes['T2DegStop_250_220.root'].columns)
print(len(dataframes['T2DegStop_250_220.root']))


Most of the variables are reconstructed properties of the event, typically properties of the reconstructed particles. There are a number of special variables:

- `BDT*`: these 8 variables are the result of applying the BDT methods from the publication to the event data, remember there are 8 different BDTs, one for each $\Delta M$, when analysing signal samples, we should only look at the corresponding BDT variable and when analysing background samples we should look at the BDT variable corresponding to the $\Delta M$ region we are analysing. For this exercise, we are considering only one region, $\Delta M = 30\,\text{GeV}$.
- `Run`, `Event`, `LumiSec`: these 3 variables uniquely identify each event, and are not really important for this exercise
- `Nevt`, `genWeight`, `sumGenWeight`: are variables necessary for correctly weighing simulated events
- `XS`: is the cross section to consider for the process when calculating yields
- `weight`: is the weight to apply to the event for calculating yields

### Plotting histograms of some observables using plotly
(Plotly is built on top of matplotlib and provides a bunch of useful and visually pleasing plotting options, it also offers interactive graphs which are great when exploring your data. See examples on [plotly](https://plotly.com/python/) website)

Here we take a look at the $p_T$ distribution of the leading lepton for one of the signal samples

In [ ]:
fig = px.histogram(
    dataframes['T2DegStop_250_220.root'],
    x='LepPt',
    y='weight',
    histfunc='sum',
    title='Leading Lepton Transverse Momentum',
    labels={'LepPt': 'Lepton 1 $p_T$ [GeV]'},
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=100.0,
        size=1
    ))
fig.update_layout(yaxis_title="Weight")
fig.show()

We can compare the signal sample against one or more background samples, here we consider the leading Jet $p_T$

In [ ]:
df = pandas.concat([dataframes['T2DegStop_250_220.root'], dataframes['WJets']])
fig = px.histogram(
    df,
    x='Jet1Pt',
    y='weight',
    histfunc='sum',
    title='Leading Jet Transverse Momentum - WJets vs Signal',
    labels={'Jet1Pt': 'Jet 1 $p_T$ [GeV]'},
    color="subcategory",
    barmode="overlay", # Without this option, plotly will stack the histograms by default
    #marginal="rug",
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=800.0,
        size=4
    ))
fig.update_layout(yaxis_title="Weight")

fig.show()

del df

Each process has a different expected amount of events, which makes it hard to compare the shape, so we artificially modify the weights to get histograms with similar area:

In [ ]:
df_sig = dataframes['T2DegStop_250_220.root'].copy(deep=True)
df_sig["category"] = "Signal_250_220"
df_back = dataframes['WJets'].copy(deep=True)
df_back["category"] = "WJets"

df_sig["weight"] = df_sig["weight"]*(df_back["weight"].sum())/(df_sig["weight"].sum())
df = pandas.concat([df_sig, df_back])
fig = px.histogram(
    df,
    x='Jet1Pt',
    y='weight',
    histfunc='sum',
    title='Leading Jet Transverse Momentum - WJets vs Signal',
    labels={'Jet1Pt': 'Jet 1 $p_T$ [GeV]'},
    color="category",
    barmode="overlay", # Without this option, plotly will stack the histograms by default
    #marginal="rug",
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=800.0,
        size=4
    ))
fig.update_layout(yaxis_title="Weight")
fig.show()

In [ ]:
# Some clean up
del df
del df_sig
del df_back

We should compare the different signal points against each other to verify our expectation that the events kinematics depend to first order on the $\Delta M$ quantity. Let check some properties within a $\Delta M$ region:

In [ ]:
# Build dataframe for some signals points with same DM
df_1 = dataframes['T2DegStop_250_220.root'].copy(deep=True)
df_1['weight'] = df_1['weight']/(df_1['weight'].sum())
df_2 = dataframes['T2DegStop_300_270.root'].copy(deep=True)
df_2['weight'] = df_2['weight']/(df_2['weight'].sum())
df_3 = dataframes['T2DegStop_500_470.root'].copy(deep=True)
df_3['weight'] = df_3['weight']/(df_3['weight'].sum())
df_4 = dataframes['T2DegStop_650_620.root'].copy(deep=True)
df_4['weight'] = df_4['weight']/(df_4['weight'].sum())
df_5 = dataframes['T2DegStop_800_770.root'].copy(deep=True)
df_5['weight'] = df_5['weight']/(df_5['weight'].sum())
df = pandas.concat([
    df_1,
    df_2,
    df_3,
    df_4,
    df_5,
                    ])

In [ ]:
# Plot lep PT
fig = px.histogram(
    df,
    x='LepPt',
    y='weight',
    histfunc='sum',
    title='Leading Lepton Transverse Momentum - Different DM=30 signals',
    labels={'LepPt': 'Lepton 1 $p_T$ [GeV]'},
    color="subcategory",
    barmode="overlay", # Without this option, plotly will stack the histograms by default
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=100.0,
        size=1
    ))
fig.update_layout(yaxis_title="Weight")

fig.show()

In [ ]:
# Plot mt
fig = px.histogram(
    df,
    x='mt',
    y='weight',
    histfunc='sum',
    title='Event transverse mass - Different DM=30 signals',
    labels={'mt': '$m_T$'},
    color="subcategory",
    barmode="overlay", # Without this option, plotly will stack the histograms by default
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=500.0,
        size=10
    ))
fig.update_layout(yaxis_title="Weight")

fig.show()

In [ ]:
# Clean up
del df
del df_1
del df_2
del df_3
del df_4
del df_5

Now, let's compare signal samples from different $\Delta M$ regions:

In [ ]:
# Build dataframe for some signals points with different DM
df_1 = dataframes['T2DegStop_250_240.root'].copy(deep=True)
df_1['weight'] = df_1['weight']/(df_1['weight'].sum())
df_2 = dataframes['T2DegStop_250_220.root'].copy(deep=True)
df_2['weight'] = df_2['weight']/(df_2['weight'].sum())
df_3 = dataframes['T2DegStop_250_200.root'].copy(deep=True)
df_3['weight'] = df_3['weight']/(df_3['weight'].sum())
df_4 = dataframes['T2DegStop_250_170.root'].copy(deep=True)
df_4['weight'] = df_4['weight']/(df_4['weight'].sum())
df = pandas.concat([
    df_1,
    df_2,
    df_3,
    df_4,
                    ])

In [ ]:
# Plot lep PT
fig = px.histogram(
    df,
    x='LepPt',
    y='weight',
    histfunc='sum',
    title='Leading Jet Transverse Momentum - Signals with Different DM',
    labels={'Jet1Pt': 'Jet 1 $p_T$ [GeV]'},
    color="subcategory",
    barmode="overlay", # Without this option, plotly will stack the histograms by default
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=100.0,
        size=1
    ))
fig.update_layout(yaxis_title="Weight")

fig.show()

In [ ]:
# Plot DR Jet1 Lep
fig = px.histogram(
    df,
    x='mt',
    y='weight',
    histfunc='sum',
    title='Event transverse mass - Signals with Different DM',
    labels={'mt': '$m_T$'},
    color="subcategory",
    barmode="overlay", # Without this option, plotly will stack the histograms by default
)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=500.0,
        size=10
    ))
fig.update_layout(yaxis_title="Weight")

fig.show()

In [ ]:
# clean up
del df
del df_1
del df_2
del df_3
del df_4

### Multiple scatter plots

We can also do scatter plots of variables against other variables, keep in mind that scatter plots implement one point per entry in the dataframe and so do not respect the event weighting of the MC samples, but they are stil very useful for understanding correlations in the data

In [ ]:
df = pandas.concat([dataframes['T2DegStop_250_220.root'], dataframes['ZInv'], dataframes['Diboson']])
fig = px.scatter_matrix(
    df,
    dimensions=["LepPt", "LepEta", "Met", "mt", "Q80", "HT"],
    color="subcategory"
                        )
fig.update_traces(diagonal_visible=False, showupperhalf=False,)
fig.show()
del df

In the interactive plot above we can enable and disable the different colors to inspect them individually or together, we can also lasso select some of the scatter points in one of the plots and see how those specific points behave in the other plots. This is an extremely useful tool for exploring and understanding the data.


### Plots of Subleading variables

Now let's look at one of the variables which does not belong to one of the leading objects or one of the event level quantities:

In [ ]:
fig = px.histogram(
    dataframes['WJets'],
    x='Jet2Pt',
    y='weight',
    histfunc='sum',
    title='Subleading Jet Transverse Momentum',
    labels={'Jet2Pt': 'Jet 2 $p_T$ [GeV]'},
)
# Purposefully removing setting the binning
#fig.update_traces(xbins=dict( # bins used for histogram
#        start=0.0,
#        end=100.0,
#        size=1
#    ))
fig.update_layout(yaxis_title="Weight")
fig.show()

What is up with the large peak at -10000?


.

.

.

.

.

.

.

.



The preselection of this analysis selects events with at least one lepton and at least one jet. Events with more objects of each will pass these criteria, but not all events will have multiple leptons and/or jets.

As a result, a default "unphysical" value of -9999 is attributed to events where not all objects are present. this should serve as a "pseudo-categorical" variable for any machine learning approach. However, the magnitude of the unphysical default value may have an affect on the ML performance, if you want to evaluate what the effect of using a different default value may be, use the `DataC` data directory, which uses a significantly smaller magnitude value than -10000.

We will use the approach of considering all the data, but there are other equally valid approaches, for instance we could consider only events with 1 lepton and 1 jet and eventually train additional algorithms for events with 2 leptons and 1 jet, 2 leptons and 2 jets and so on.

### Correlation matrix

Evaluating the correlation matrix of the features allows to evaluate which ones are highly correlated and identify which ones could possibly not be included in the set of input features of the ML algorithm.

What is particularly relevant in this application is the correlations in the different classes. If a pair of features are highly correlated in one class and not correlated in another, this information can be exploited by the ML algorithm to better separate the classes.

We will be using the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), which only captures linear correlations, so the figure below is not a complete picture

In [ ]:
import seaborn as sns
sns.set()

def corrmatrix(corr, label):
    plt.figure()
    ax = sns.heatmap(
        corr,
        vmin=-1., vmax=1., center=0.,
        cmap=sns.diverging_palette(20., 220., n=200, as_cmap=True),
        square=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    );

    ax.set_title('Correlation matrix for %s events' % label)

    plt.show()
    plt.close()


drop_cols = [
    'Run',
    'Event',
    'LumiSec',
    'Nevt',
    'XS',
    'nVert',
    'weight',
    'genWeight',
    'sumGenWeight',
    'BDT10',
    'BDT20',
    'BDT30',
    'BDT40',
    'BDT50',
    'BDT60',
    'BDT70',
    'BDT80',
    'isTrain',
    'category',
    'subcategory',
]

corrmatrix(dataframes["DM30"].drop(drop_cols, axis=1).corr(), 'signal')
corrmatrix(dataframes["Background"].drop(drop_cols, axis=1).corr(), 'background')

## Prepare data for ML

For training the ML algorithm we will not use all the MC background processes, but only a selection of the main processes. This includes $t\bar{t}$ and WJets processes. There are two separate simulated MC samples for $t\bar{t}$ and the ZInv MC sample can also optionally be included. This gives a few different options for choosing on what to train the model, if you want to try a different option, scroll up and change the relevant line in the load dataframe function.

We will also have to create a label for signal or background events, which will serve as the target for the ML algorithm and join the datafraes together. We will also drop a number of the features, keeping only those which we want the ML algorithm to train on. In order to maintain parity with the paper, we will use by default the same set as there, but please consider trying different features to see if you are able to improve upon the paper results.

In [ ]:
# Create a new column 'label' and set its value to 1 or 0 for all rows (=events)
sig = dataframes['DM30'].copy(deep=True)
bkg = dataframes['Background'].copy(deep=True)
sig['label'] = 1
bkg['label'] = 0

# Merge the signal and background into one dataframe
print(f"Signal shape {sig.shape}")
print(f"Bkg shape {bkg.shape}")

data = pandas.concat([sig,bkg])

print(f"Data shape {data.shape}")
print(data.columns)

# Drop unneeded features, If you are using a different DM region, keep the BDT of your DM here instead of BDT30
data = data[["Jet1Pt","mt","Met","LepChg","LepEta","LepPt","HT","NbLoose","Njet","JetHBpt","DrJetHBLep","JetHBCSV","BDT30","weight","label"]]

print(f" Data shape {data.shape}")
print(data.columns)
print(f"In this dataframe we finally have {data[data['label']==1].shape[0]} signal and {data[data['label']==0].shape[0]} background events")

This data set is still ordered, ie. each signal point comes after each other, followed by the background events, first $t\bar{t}$ and then WJets. ML training requires a shuffled data set instead.

It is good practice to shuffle the data as soon as possible even though shuffling can be done at the time of splitting into the training and testing datasets.

We will also separate features and labels from each other, and check for corrupted values.

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

In [ ]:
print("There are NaN-filled elements:", data.isna().any().any())

X = data.drop(["label"], axis=1)
y = data["label"]

print(f"data shape {data.shape}")
print(f"input feature shape {X.shape}")
print(f"label (=target) shape {y.shape}")


### Split the data set into training and test set


Typically, in a ML approach, there should be 3 independent datasets, a training dataset on which the ML algorithm is trained; a testing dataset on which the ML algorithm is tested and whose results are used to inform on hyperparameter tuning and other optimizations; and an application dataset, which is the data we are really interested in studying.

Often, the test and application datasets are merged, particularly when there is no tuning of the hyperparameters, but we should always be aware of this shortcut since it may lead to biases. This is what we will do here.

For full compatibility with the paper results, we should also use the same splitting as was used in the paper, unfortunately, this data has been challenging to recover, so we will define our own splitting and hope that the results obtained here still translate and compare favourably to those of the paper.

![Example sample splitting](figs/trainingNetwork.png)

(Image: P. Vischia, [doi:10.5281/zenodo.6373442](https://doi.org/10.5281/zenodo.6373442))

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

print(f"We have {len(X_train)} training samples with {sum(y_train)} signal and {sum(1-y_train)} background events")
print(f"We have {len(X_test)} testing samples with {sum(y_test)} signal and {sum(1-y_test)} background events")


So far we have purposefully kept the BDT output, from the paper, attached as a feature of the data as well as the event weight. We should now remove it to avoid feeding it into the ML algorithm.

In [ ]:
BDT_train = X_train["BDT30"]
BDT_test = X_test["BDT30"]
weight_train = X_train["weight"]
weight_test = X_test["weight"]

X_train.drop(["BDT30", "weight"], axis=1, inplace=True)
X_test.drop(["BDT30", "weight"], axis=1, inplace=True)

### Preprocess the data
Before training a first ML-based classifier we need to think about if any preprocessing of the data is required. Many ML algorithms are based on gradient minimization techniques that can fail if the inputs have numbers that widely-vary in magnitude (remember the default "unphysical" value of -9999). For example, we saw the leading Jet $p_T$ covers a range of several orders of magnitude, which can prevent the convergence of a minimization algorithm.

There are several scaling methods offered in the ML libraries, each method has its own advantages and drawbacks. Another possibility is to use PCA, which can result in a more targetted set of input features. Below we use the Standard Scaler by default, but import a few of the other options for you to experiment with.

We will also store the original train and test structures, in case you ever need to revert.


In [ ]:
X_train_orig = X_train.copy()
y_train_orig = y_train.copy()
X_test_orig = X_test.copy()
y_test_orig = y_test.copy()

from sklearn.preprocessing import (
    MaxAbsScaler, # maxAbs
    MinMaxScaler, # MinMax
    Normalizer, # Normalization (equal integral)
    StandardScaler# standard scaling
)
from sklearn.decomposition import PCA

# Scale the input features and the target variable
for column in X_train.columns:
    scaler = StandardScaler().fit(X_train.filter([column], axis=1))
    X_train[column] = scaler.transform(X_train.filter([column], axis=1))
    X_test[column] = scaler.transform(X_test.filter([column], axis=1))



## Train a dense neural network


For this exercise we will use `pytorch`, a backend designed natively for tensor operations.

You could also use one of the other machine learning libraries available, such as the `tensorflow` backend, either directly or through the `keras` frontend.

`torch` handles the data management via the `Dataset` and `DataLoader` classes.
Here we don't need any specific `Dataset` class, because we are not doing sophisticated things, but you may need that in the future.

The `DataLoader` class takes care of providing quick access to the data by sampling batches that are then fed to the network for (mini)batch gradient descent.


Set a manual seed, for reproducibility

In [ ]:
torch.manual_seed(42)

In [ ]:
class StopDataset(Dataset):
    def __init__(self, X, y, device=torch.device("cpu")):
        self.X = torch.Tensor(X.values if isinstance(X, pandas.core.frame.DataFrame) else X).to(device)
        self.y = torch.Tensor(y.values).to(device)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        label = self.y[idx]
        datum = self.X[idx]

        return datum, label

batch_size=512 # Minibatch learning


train_dataset = StopDataset(X_train, y_train)
test_dataset = StopDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


As an example of the DataLoader functionality, let's access the data loader via its iterator, and sample a single batch by calling `next` on the iterator

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
train_features

Let's build a simple neural network, by inheriting from the `nn.Module` class. **This is very crucial, because that class is the responsible for providing the automatic differentiation infrastructure for tracking parameters and performing backpropagation**

In [ ]:
class StopNeuralNetwork(nn.Module):
    def __init__(self, ninputs, device=torch.device("cpu")):
        super().__init__()
        self.device = device

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(ninputs, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
        self.linear_relu_stack.to(device)

    def forward(self, x):
        # Pass data through conv1
        x = self.linear_relu_stack(x)
        return x

Let's instantiate the neural network and print some info on it

In [ ]:
model = StopNeuralNetwork(X_train.shape[1])

print(model) # some basic info

print("Now let's see some more detailed info by using the torchinfo package")
torchinfo.summary(model, input_size=(batch_size, X_train.shape[1])) # the input size is (batch size, number of features)

In [ ]:
train_dataset = StopDataset(X_train, y_train, device=device)
test_dataset = StopDataset(X_test, y_test, device=device)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

random_batch_X, random_batch_y = next(iter(train_dataloader))

# Reinstantiate the model, on the chosen device
model = StopNeuralNetwork(X_train.shape[1], device)

#check if the NN can be evaluated some data; note: it has not been trained yet
print (model(torch.tensor(X_train.values[:10],device=device)))

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler, best_model_path, device):
    size = len(dataloader.dataset)
    losses=[] # Track the loss function
    # Set the model to training mode - important for batch normalization and dropout layers
    model.train()
    best_loss = np.inf
    for (X,y) in tqdm(dataloader):
        # Reset gradients (to avoid their accumulation)
        optimizer.zero_grad()
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred.squeeze(dim=1), y)
        losses.append(loss.detach().cpu())
        if loss < best_loss:
            best_loss = loss.detach().cpu()
            torch.save(model.state_dict(), best_model_path) # Save the full state of the model, to have access to the training history
        # Backpropagation
        loss.backward()
        optimizer.step()

    scheduler.step()
    return np.mean(losses)

Now we need to define the loop that is run on the test dataset.

**The test dataset is just used for evaluating the output of the model. No backpropagation is needed, therefore backpropagation must be switched off!!!**

In [ ]:
def test_loop(dataloader, model, loss_fn, device):
    losses=[] # Track the loss function
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for (X,y) in tqdm(dataloader):
            pred = model(X)
            loss = loss_fn(pred.squeeze(dim=1), y).item()
            losses.append(loss)

    return np.mean(losses)

We are now ready to train this network!

Torch provides the functionality to use generic functions as loss function. Since we are trying to do classification, we will set our loss function to be the cross entropy.

In [ ]:
epochs=30
learningRate = 0.01

# The loss defines the metric deciding how good or bad is the prediction of the network
loss_fn = torch.nn.BCELoss()
# The optimizer decides which path to follow through the gradient of the loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
# The scheduler reduces the learning rate for the optimizer in order for the optimizer to be able to "enter" narrow minima
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


In [ ]:
train_losses=[]
test_losses=[]
best_model_path = "best_stop_dnn_model.h5"
for t in range(epochs):
    print(f"Epoch {t+1}/{epochs}\n-------------------------------")
    train_loss=train_loop(train_dataloader, model, loss_fn, optimizer, scheduler, best_model_path, device)
    test_loss=test_loop(test_dataloader, model, loss_fn, device)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print("Avg train loss", train_loss, ", Avg test loss", test_loss, "Current learning rate", scheduler.get_last_lr())
print("Done!")

We can now plot the loss evolution over the training epochs to monitor the training progression

In [ ]:
plt.figure()
plt.plot(train_losses, label="Average training loss")
plt.plot(test_losses, label="Average test loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="best")
plt.show()
plt.close()

We can now plot the Receiver Operating Characteristic (ROC) curve of the Neural network model to get an evaluation of the performance of this approach. We can also use the BDT data from the paper to print the ROC curve of the BDT approach to get a preliminary comparison between the methods.

In [ ]:
def plot_rocs(scores_labels_names):
    plt.figure()
    for score, label, weights, name  in scores_labels_names:
        fpr, tpr, thresholds = roc_curve(label, score, sample_weight=weights)
        plt.plot(
            fpr, tpr,
            linewidth=2,
            label=f"{name} (AUC = {100.*auc(fpr, tpr): .2f} %)"
        )
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.grid()
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic curve")
    plt.legend(loc="lower right")
    plt.show()
    plt.close()

plot_rocs([
    (model(torch.tensor(X_train.to_numpy(),device=model.device)).numpy(force=True), y_train, weight_train, "Train"),
    (model(torch.tensor(X_test.to_numpy(),device=model.device)).numpy(force=True), y_test, weight_test, "Test"),
    (BDT_test, y_test, weight_test, "BDT")
])

The results above seem to indicate that the neural network approach can bring some improvement to the BDT approach used in the paper.

A more rigorous evaluation should be performed, where a final selection is decided upon and the expected limits are computed.

### Model Evaluation with pyhf

We will evaluate the model performance with pyhf and compare the BDT approach from the paper with the NN approach used here. We will use some simplifications, such as ignoring all background processes not in the training background sample (we leave it as an exercise to implement the other background samples to any interested individual, the data has been loaded and is available, you just need to apply the trained NN to the required samples). We will also assume an integrated luminsoty of $35.866\,fb^{-1}$ for a more direct comparison to the results from the paper. We also assume that of the 23 signal samples contributing to the $\Delta M$ sample that they all contribute equally, i.e. we average the cross sections across the samples. This is for ease of computation and because we need to choose a single reference cross section for performing the calculations.

In [ ]:
import pyhf
pyhf.set_backend("numpy")
from pyhf.contrib.viz import brazil

luminosity = 35866

#### BDT Approach

In [ ]:
# BDT final selection
sig_yield = weight_test.loc[(BDT_test > 0.47) & (y_test == 1)].sum() * luminosity/(0.33*23)  #Divide by 0.33 to compensate for the splitting factor
bkg_yield = weight_test.loc[(BDT_test > 0.47) & (y_test == 0)].sum() * luminosity/0.33
# cuts (adjust to the signal DM you used):
#  DM10 : 0.31
#  DM20 : 0.39
#  DM30 : 0.47
#  DM40 : 0.48
#  DM50 : 0.45
#  DM60 : 0.50
#  DM70 : 0.46
#  DM80 : 0.44

print(f"Signal yield: {sig_yield}")
print(f"Background yield: {bkg_yield}")


stat_model = pyhf.simplemodels.uncorrelated_background(
    signal=[sig_yield], bkg=[bkg_yield], bkg_uncertainty=[bkg_yield*0.2] # Assume 20% uncertainty on background
)
data = [bkg_yield] + stat_model.config.auxdata

poi_vals = np.linspace(0, 3, 31)
results = [
    pyhf.infer.hypotest(
        test_poi, data, stat_model, test_stat="qtilde", return_expected_set=True
    )
    for test_poi in poi_vals
]

fig, ax = plt.subplots()
fig.set_size_inches(7, 5)
brazil.plot_results(poi_vals, results, ax=ax)
fig.show()

#### NN Approach

For the NN approach, we need to choose a cut value, select the cut value which maximizes FOM

In [ ]:
# Maximize FOM, for now use simple S/sqrt(B)
from math import sqrt
from operator import itemgetter
NN_test = model(torch.tensor(X_test.to_numpy(),device=model.device)).numpy(force=True)
cut_vals = np.linspace(-1, 1, 21)
FOM_vals = []
for cut in cut_vals:
  S = weight_test.loc[(NN_test > cut) & (y_test == 1)].sum() * (luminosity/(0.33*23))
  B = weight_test.loc[(NN_test > cut) & (y_test == 0)].sum() * (luminosity/0.33)
  FOM = S/sqrt(B)
  FOM_vals.appen((cut, FOM))

NN_cut = max(FOM_vals, key=itemgetter(1))[0]
NN_cut

In [ ]:
# BDT final selection

sig_yield = weight_test.loc[(NN_test > NN_cut) & (y_test == 1)].sum() * luminosity/(0.33*23)  #Divide by 0.33 to compensate for the splitting factor
bkg_yield = weight_test.loc[(NN_test > NN_cut) & (y_test == 0)].sum() * luminosity/0.33

print(f"Signal yield: {sig_yield}")
print(f"Background yield: {bkg_yield}")

stat_model = pyhf.simplemodels.uncorrelated_background(
    signal=[sig_yield], bkg=[bkg_yield], bkg_uncertainty=[bkg_yield*0.2] # Assume 20% uncertainty on background
)
data = [bkg_yield] + model.config.auxdata

poi_vals = np.linspace(0, 3, 31)
results = [
    pyhf.infer.hypotest(
        test_poi, data, model, test_stat="qtilde", return_expected_set=True
    )
    for test_poi in poi_vals
]

fig, ax = plt.subplots()
fig.set_size_inches(7, 5)
brazil.plot_results(poi_vals, results, ax=ax)
fig.show()

## Train an autoencoder neural network

TODO

